In [5]:
from utilities.grover_state_preparation import *
from utilities.auxiliaries import *
from config import *
     
import numpy as np
import pandas as pd
from scipy.stats import beta, wasserstein_distance
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from qiskit import transpile
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

In [6]:
p_i_set = beta.pdf(np.linspace(a, b, 2**m), alpha_, beta_) 
p_i_set /= p_i_set.sum()
n_angles = len(get_grover_angles(p_i_set, m))

In [7]:
results = pd.DataFrame(columns=["trainable angles", "best loss", "Earth Mover's Distance"])

In [8]:
for k in range(1, n_angles + 1):  
    for idx_thetas_to_optimize in combinations(range(n_angles), k):
        idx_thetas_to_optimize = list(idx_thetas_to_optimize)

        thetas_to_optimize = np.array(generate_parameters(len(idx_thetas_to_optimize), k=2))
        thetas = get_grover_angles(p_i_set, m)

        print(f"\nTesting configuration: {idx_thetas_to_optimize}")  

        best_loss = float('inf')
        best_thetas = None

        for run in range(runs):
            result = minimize(objective_function, thetas_to_optimize,
                              args=(idx_thetas_to_optimize, thetas, p_i_set, shots),
                              method=optimizer_type, options={"disp": False, "maxiter": max_iterations})
            
            current_loss = result.fun
            thetas_to_optimize = result.x
            
            if current_loss < best_loss:
                best_loss = current_loss
                best_thetas = np.array(thetas.copy())
                best_thetas[idx_thetas_to_optimize] = thetas_to_optimize 
        
        final_qc  = state_expansion(m, best_thetas)
        t_qc = transpile(final_qc, backend=backend)
        job = backend.run(t_qc, shots=shots)
        counts = job.result().get_counts(final_qc)
        samples = np.array([counts.get(state, 0) for state in all_states], dtype=float)
        samples /= samples.sum()

        
        emd = wasserstein_distance(samples, p_i_set)

        
        print(f"Configuration {idx_thetas_to_optimize} -> Best Loss: {best_loss:.10f}, EMD: {emd:.10f}")


        

        new_row = pd.DataFrame([{
            "trainable angles": str(idx_thetas_to_optimize),
            "best loss": best_loss,
            "Earth Mover's Distance": emd
        }])
        results = pd.concat([results, new_row], ignore_index=True)

In [ ]:
saving_folder = r"..." #change this according to your prefered saving folder
excel_filename = os.path.join(saving_folder, "testing_m3.xlsx") #change this according to your prefered file saving name
results.to_excel(excel_filename, index=False)
print(f"\nResults saved at: '{excel_filename}'")